In [1]:
using Revise

using RigidBodyDynamics
using RigidBodyDynamics: Bounds

using Plots
using BilevelTrajOpt

In [2]:
urdf = joinpath("..", "urdf", "ball.urdf")
mechanism = parse_urdf(Float64, urdf)
body = findbody(mechanism, "ball")
basejoint = joint_to_parent(body, mechanism)
floatingjoint = Joint(basejoint.name, frame_before(basejoint), frame_after(basejoint), QuaternionFloating{Float64}())
replace_joint!(mechanism, basejoint, floatingjoint)
position_bounds(floatingjoint) .= Bounds(-100, 100)
velocity_bounds(floatingjoint) .= Bounds(-100, 100)
μ = 0.5
motion_type = :xyz
point = Point3D(default_frame(findbody(mechanism, "floor")), [0.,0.,0.])
normal = FreeVector3D(default_frame(findbody(mechanism, "floor")), [0.,0.,1.])
hs = HalfSpace(point, normal)
floor = Obstacle(hs, μ, motion_type)
obstacles = [floor]
env = parse_contacts(mechanism, urdf, obstacles)
x0 = MechanismState(mechanism)
Δt = 0.005
sim_data = get_sim_data(x0,env,Δt,true);

In [3]:
q0 = [1., 0., 0., 0., 0., 0., 0.]
v0 = [0., 0., 0., 0.1, 0.1, 0.]

u0 = zeros(sim_data.num_v)
set_configuration!(x0, q0)
set_velocity!(x0, v0)
setdirty!(x0)
ctrl! = (u,t,x) -> u[:] = 0.
traj = BilevelTrajOpt.simulate(x0,env,sim_data.Δt,1,ctrl!,implicit_contact=false)
qnext = traj[1:sim_data.num_q,2]
vnext = traj[sim_data.num_q+1:sim_data.num_q+sim_data.num_v,2]
x_sol_exp = traj[sim_data.num_q+sim_data.num_v+2:end,2]

# qnext = q0
# vnext = v0

τ_ip, x_sol_ip, λ_ip, μ_ip, fx_ip = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=true);

τ_auglag, x_sol_auglag, λ_auglag, μ_auglag, fx_auglag = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=false);

display("Explicit")
display(x_sol_exp)
display("IP")
display(x_sol_ip)
display("Aug Lag")
display(x_sol_auglag)

display("torques")
display(τ_ip)
display(τ_auglag)

elapsed time: 23.677732814 seconds
elapsed time: 0.004962273 seconds

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

elapsed time: 0.004321039 seconds
elapsed time: 0.005525651 seconds
elapsed time: 0.004324706 seconds
elapsed time: 0.004271795 seconds
elapsed time: 0.004239183 seconds
elapsed time: 0.004366555 seconds
elapsed time: 0.004574319 seconds
elapsed time: 0.004282986 seconds
elapsed time: 0.004291372 seconds
elapsed time: 0.004291222 seconds
elapsed time: 0.004371003 seconds
elapsed time: 0.004340285 seconds
elapsed time: 0.004233302 seconds
elapsed time: 0.004416279 seconds
elapsed time: 0.004483446 seconds
elapsed time: 0.004345175 seconds

"Explicit"

6-element Array{Float64,1}:
 5.58841e-8
 5.58841e-8
 0.24999   
 0.24999   
 0.087738  
 0.999959  

"IP"

6-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

...

6-element Array{Float64,1}:
 1.77679e-8
 1.85149e-8
 0.24999   
 0.24999   
 0.087738  
 0.999959  

"torques"

6-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

6-element Array{Float64,1}:
  0.0   
  0.0   
  0.0   
  2.4524
  2.4524
 -9.8096

Not_Enough_Degrees_Of_Freedom


In [4]:
J_auto = ForwardDiff.jacobian(qv -> solve_implicit_contact_τ(sim_data,q0,v0,u0,qv[1:sim_data.num_q],qv[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=false)[2], vcat(qnext,vnext))

6×13 Array{Float64,2}:
 0.0  -0.0135725    0.0135724   …    1.10736    1.1072    0.553617  
 0.0  -0.0135718    0.0135719        1.10719    1.10724   0.553591  
 0.0  -0.100423     0.100504       -12.195      8.19231   4.0962    
 0.0  -0.1005       0.100419         8.19223  -12.1949    4.09606   
 0.0   1.99951e-6  -1.99974e-6       0.5        0.5       9.51224e-9
 0.0  -0.456549     0.456549    …   -3.54255   -3.54245  18.6159    

In [11]:
τ_auglag, x_sol_auglag, λ_auglag, μ_auglag, fx_auglag = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=false);

# autodiff 
J_auto = ForwardDiff.jacobian(qv -> solve_implicit_contact_τ(sim_data,q0,v0,u0,qv[1:sim_data.num_q],qv[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=false)[2], vcat(qnext,vnext))
# Jt_auto = ForwardDiff.jacobian(qv -> solve_implicit_contact_τ(sim_data,q0,v0,u0,qv[1:sim_data.num_q],qv[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=false)[1], vcat(qnext,vnext))

# numerical
ϵ = 1e-8
J_num = zeros(size(J_auto))
# Jt_num = zeros(size(J_auto))
# Jt_num_ip = zeros(size(J_auto))
qv = vcat(qnext,vnext)
for i = 1:length(qv)
    δ = zeros(length(qv))
    δ[i] = ϵ 
    τ, x, λ, μ, fx = solve_implicit_contact_τ(sim_data,q0,v0,u0,(qv+δ)[1:sim_data.num_q],(qv+δ)[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=false)
    J_num[:,i] = (x - x_sol_auglag)/ϵ
#     Jt_num[:,i] = (τ - τ_auglag)/ϵ
#     τ, x, λ, μ, fx = solve_implicit_contact_τ(sim_data,q0,v0,u0,(qv+δ)[1:sim_data.num_q],(qv+δ)[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=true)
#     Jt_num_ip[:,i] = (τ - τ_auglag)/ϵ
end

display(J_auto)
display(J_num)

# display(Jt_auto)
# display(Jt_num)
# display(Jt_num_ip)

6×13 Array{Float64,2}:
 0.0  -0.0135775    0.0135775   …    1.1072      1.1072     0.553618 
 0.0  -0.0135771    0.0135771        1.1072      1.1072     0.553602 
 0.0  -0.100459     0.100459       -12.195       8.19238    4.09618  
 0.0  -0.100459     0.100459         8.19237   -12.195      4.09618  
 0.0   8.72873e-5  -8.72174e-5       0.500858    0.500858  -0.0035544
 0.0  -0.456558     0.456558    …   -3.54258    -3.54258   18.616    

6×13 Array{Float64,2}:
 0.0  -0.0185299     0.00553914    0.00044324   …    1.11388    0.548876   
 0.0  -0.0190978     0.00923254   -0.00305802        1.1071     0.546554   
 0.0  -0.0981233     0.105992     -0.49522           8.19176    4.09663    
 0.0  -0.0945507     0.10802       0.501013        -12.1943     4.1035     
 0.0   0.000206446  -0.000168009   0.000576633       0.499467   0.000207435
 0.0  -0.458721      0.458196      0.00693677   …   -3.53242   18.6096     

In [36]:
display(J_auto)
display(J_num)

6×13 Array{Float64,2}:
 0.0  -2.66371e-9    0.041909     …    1.70883  -2.55858e-6   0.854414  
 0.0  -1.7208e-8     0.0838183         3.41767   4.66777      1.70883   
 0.0  -2.65416e-8    0.240365        -10.5865   -1.46727e-5   4.90042   
 0.0   7.27417e-9    0.0838181         3.41766  -4.66774      1.70883   
 0.0   1.12996e-10  -3.87681e-10       1.0      -5.35341e-9  -3.94319e-8
 0.0  -7.98513e-8    0.900771     …   -4.04601   6.06338e-6  18.3644    

6×13 Array{Float64,2}:
 0.0  -0.000943053   0.0377401    -0.00695856  …  -0.0059646    0.842861   
 0.0   0.00141541    0.0841931    -0.23106         4.67418      1.7125     
 0.0  -0.00397753    0.233688      0.0137627       0.00101429   4.89938    
 0.0   0.00480142    0.0861786     0.2209         -4.66442      1.71835    
 0.0  -0.000216623  -0.000396977   0.00192992      0.00187081  -0.000515939
 0.0   0.000233757   0.886196     -0.00758604  …   0.00823586  18.3675     

In [40]:
display(J_auto[:,5:10])
display(J_num[:,5:10])
# display(Jt_num_ip[:,11:13])

6×6 Array{Float64,2}:
 0.0  0.0  -2.05647e-10   2.19791e-16  -0.000322436  -9.83194e-11
 0.0  0.0  -4.13323e-10  -2.66586e-11  -0.000644871   0.0017615  
 0.0  0.0  -1.20477e-9    6.16206e-15  -0.0018493    -5.80304e-10
 0.0  0.0  -4.10356e-10   2.66481e-11  -0.00064487   -0.0017615  
 0.0  0.0  -4.24756e-13   1.87721e-17   6.0039e-12   -9.83364e-12
 0.0  0.0  -4.4723e-9    -8.34799e-15  -0.00693025    2.2827e-10 

6×6 Array{Float64,2}:
 0.0  0.0  -0.00435328   -0.00926907   -0.0070735    -0.00340542 
 0.0  0.0   0.00434015   -0.00327026    0.00381036    0.00741166 
 0.0  0.0  -0.00288652    0.00878751   -0.00310972   -0.00292666 
 0.0  0.0  -0.00740216   -0.000392491   0.00351041   -0.0013809  
 0.0  0.0   0.000146949   0.000392512  -0.000396957   0.000250959
 0.0  0.0  -0.0231815    -0.00738357   -0.00305753   -0.000421307

In [ ]:
 (J_num[:,1] - x_sol_auglag)/ϵ

In [63]:
maximum(abs.(J_auto-J_num))

0.027098530150311717

In [ ]:
abs.(J_auto-J_num)

In [ ]:
J_num

In [ ]:
J_auto